In [ ]:
import pandas as pd

file_path = 'train.csv'
train_data = pd.read_csv(file_path)

train_data.head()


,metaphorID,label_boolean,text
0,0,True,"Hey , Karen !!!! I was told that on the day of..."
1,2,False,"Hi Ladies ... my last chemo was Feb 17/09 , ra..."
2,2,False,I have just come form my consult with a lovely...
3,4,False,I also still question taking Tamox for stage 1...
4,2,False,Just checking in to say hello ladies . I had a...


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U



```
# This is formatted as code
```

Logestic Regression

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

train_file_path = 'train.csv'
df_train = pd.read_csv(train_file_path)

df_train_shuffled = df_train.sample(frac=1, random_state=42)
train_set, test_set = train_test_split(df_train_shuffled, test_size=0.2, random_state=42)
test_set.to_csv('test.csv', index=False)

test_file_path = 'test.csv'
df_test = pd.read_csv(test_file_path)


def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text

df_train['text'] = df_train['text'].apply(normalize_text)
df_test['text'] = df_test['text'].apply(normalize_text)

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_text = vectorizer.fit_transform(df_train['text'])
X_test_text = vectorizer.transform(df_test['text'])

onehot_encoder = OneHotEncoder()
X_train_metaphorID = onehot_encoder.fit_transform(df_train[['metaphorID']])
X_test_metaphorID = onehot_encoder.transform(df_test[['metaphorID']])

X_train = hstack([X_train_text, X_train_metaphorID])
X_test = hstack([X_test_text, X_test_metaphorID])
y_train = df_train['label_boolean']
y_test = df_test['label_boolean']


model = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

pd.DataFrame({"actual": y_test, "predicted": y_pred}).to_csv("logistic_regression_improved_predictions.csv", index=False)


Accuracy: 0.9358288770053476
Precision: 0.9962825278810409
Recall: 0.9209621993127147
F1 Score: 0.9571428571428572
